In [1]:
#library import
import pandas as pd
import numpy as np

In [2]:
#loading DataSet
data = pd.read_csv('energydata.csv')

In [3]:
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


## Question 1

A best fit line relating X and Y has a R-Squared value of 0.75. How do I interpret this information?

#### Answer = 75% of the variance in Y is explained by X 

## Question 2

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [4]:
data.drop(['date', 'lights'], axis=1, inplace=True)

In [5]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
features = normalised.drop([ 'Appliances'], axis=1)
target = normalised[ 'Appliances' ]

In [6]:
#Now, we split our dataset into the training and testing dataset. Recall that we
#had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target,
test_size= 0.3 , random_state= 42 )

In [7]:
#Feature Selection and Lasso Regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha= 0.001 )
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [8]:
def get_weights_df(lasso_model, feat, col_name) :
#this function returns the weight of every feature
    weights = pd.Series(lasso_model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = [ 'Features' , col_name]
    weights_df[col_name].round( 3 )
    return weights_df

In [9]:
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight' )
lasso_weights_df 

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


#### Answer = 4

## Question 3

The Lasso can be interpreted as least-squares linear regression where:

#### Answer = Weights are regularized with the L1 norm

## Question 4

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [10]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
def get_weights_df(linear_model, feat, col_name) :
#this function returns the weight of every feature
    weights = pd.Series(linear_model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = [ 'Features' , col_name]
    weights_df[col_name].round( 3 )
    return weights_df

In [12]:
linear_model_weights = get_weights_df(linear_model,x_train, 'Linear_Model_Weight' )
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


#### Answer = RH_2 and RH_1 Respectively

## Question 5

In the different terms of the bias-variance tradeoff, which of the following is substantially more harmful to the test error than the training error?

#### Answer = Variance

## Question 6

What is the Residual Sum of Squares (in two decimal places)?

In [13]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [14]:
#Residual Sum of Squares (RSS)
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2 ) 

45.35

#### Answer = 45.35

## Question 7

How many coefficients do you need to estimate a simple linear regression model (One independent variable)?

#### Answer = 1

## Question 8

What is the Coefficient of Determination (in two decimal places)? 

#### Answer = 0.15

## Question 9

In linear regression, L2 regularization is equivalent to imposing a:

#### Answer = Gaussian prior

## Question 10

What is the Mean Absolute Error (in two decimal places)?

In [15]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2) 

0.05

#### Answer = 0.05

## Question 11

What is the new RMSE with the Lasso Regression (in 3 decimal places)?

In [16]:
#Feature Selection and Lasso Regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha= 0.001 )
lasso_reg.fit(x_train, y_train)
predicte = lasso_reg.predict(x_test)
#Root Mean Square Error (RMSE)
from sklearn.metrics import mean_squared_error
rm = np.sqrt(mean_squared_error(y_test, predicte))
round(rm, 3 )

0.094

#### Answer = 0.094

## Question 12

Which of these is not an assumption of Linear Regression?

#### Answer =Heteroscedasticit and Multivariate normality

## Question 13

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [17]:
#Ridge Regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha= 0.4 )
ridge_reg.fit(x_train, y_train)
predicted_value = ridge_reg.predict(x_test)
#Root Mean Square Error (RMSE)
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(y_test, predicted_value))
round(rms, 3)

0.088

#### Answer = NO

## Question 14

What can you use to find the best fit line for data in Linear Regression?

#### Answer = Least Square Error

## Question 15

Adding more basis functions in a linear model:

####  Answer = Decreases model bias

## Question 16

Cross validation:

#### Answer = Is often used to select hyperparameters

## Question 17

What is the Root Mean Squared Error (in three decimal places)?

In [18]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)
#Root Mean Square Error (RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3 )

0.088

#### Answer = 0.088

## Question 18

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [19]:
xe=data['T2'].values.reshape(-1, 1)
ye=data['T6'].values.reshape(-1, 1)
x_tr, x_te, y_tr, y_te = train_test_split(xe,ye,test_size=0.3,random_state=2)

In [20]:
regressor=LinearRegression()
regressor.fit(x_tr,y_tr)
y_p=regressor.predict(x_te)
from sklearn.metrics import r2_score
r2_sco = r2_score(y_te, y_p)
round(r2_sco, 2 ) 

0.65

#### Answer = 0.64

## Question 19

Ridge regression:

#### Answer = Reduces variance at the expense of higher bias

## Question 20

Which of the following sentence is true about outliers in Linear Regression:

#### Answer = Linear regression is sensitive to outlieres